## Chunked queries: Efficient Data Retrieval

When dealing with large result sets, fetching all the data at once can lead to excessive memory usage and slower performance. 
Use "chunked queries" to retrieve data in smaller chunks. 

Chunked queries are handy when working with millions of data points. 
Rather than requesting the entire result set in one call, we can specify a chunk size to split the data.

It's important to note that the optimal chunk size may vary depending on the specific query. Also note that while smaller chunks generally result in faster queries, setting the chunk size too small can introduce overhead by generating many requests to the database. 

In [ ]:
import pandas as pd
from lsst_efd_client import EfdClient

client = EfdClient("usdf_efd")

To enable chunked responses in your query, you can simply set the chunked parameter to `True`. 
Additionally, you can specify the `chunk_size` parameter, which determines the maximum number of data points in each chunk. 
By default, the chunk size is set to 1000, but you can adjust it based on your specific requirements.

Let's consider an example where we need to query the `lsst.sal.MTM1M3.forceActuatorData` topic. 

In [ ]:
await client.get_schema("lsst.sal.MTM1M3.forceActuatorData")


This particular topic contains a massive payload, making it inefficient to retrieve all the data using a `SELECT * FROM ...` statement. 
Instead, it is recommended to select only the fields of interest. 
This significantly improves the query speed.

The topic schema reveals the presence of array fields. 
In this topic, each element within the array corresponds to a distinct force actuator. 
Specifically, there are 156 actuators responsible for exerting forces in the `x` direction.

In [ ]:
query = """SELECT /xForce/ 
    FROM "lsst.sal.MTM1M3.forceActuatorData" 
    WHERE time > now()-6h"""
query

By implementing chunked queries with the appropriate configuration, we can retrieve a dataframe with millions of data points in less than a minute.

In [ ]:
chunks = await client._influx_client.query(query, chunked=True, chunk_size=1000)
df = pd.concat([chunk async for chunk in chunks])

In [ ]:
df.head()

In [ ]:
df.size